<a href="https://colab.research.google.com/github/Mikko812/Colab/blob/master/MLB_King_of_Covers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta as td
from google.colab import files
import json
import os
#import numpy as np

pd.set_option('display.max_rows', None) 
pd.set_option('max_colwidth', 220)
pd.options.mode.chained_assignment = None
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}
path="/content/drive/My Drive/MLB. King of Covers/"

www='https://www.covers.com'
www2='https://contests.covers.com'

In [ ]:
# Чтение таблицы
def get_stats(link):
  tab = pd.read_html(www2+link)
  return [tab[0].loc[0,'Units'], tab[0].loc[1,'Units']]

r=requests.get('https://contests.covers.com/kingofcovers/overallleaderboard/fullleaderboard/fe6d67c9-6939-4d84-8ca0-acd6010e9252?seasonPhase=RegularSeason', headers=headers)
s=BeautifulSoup(r.text, 'html.parser')
table=s.find_all('tr')
cols=table[0].get_text().split('\n')[1:-1]+['M/L Picks', 'O/U Picks']
mlb_picks=pd.DataFrame(columns=cols, index=range(50))
rows=[]
for r in range(1,51):
  row=table[r].get_text().split('\n')
  row=[x.replace('\r','').strip() for x in row if x not in ['', '\r']][:-1]
  if row[-1]!='Pending':
    row.append('View')
    row.append('Not Pending')
  profile_link=table[r].find_all('td')[-2].find('a').get('href')
  row[-2]=profile_link
  row += get_stats(profile_link)
  mlb_picks.loc[r-1,:]=row
mlb_picks
pending=mlb_picks.query("Status!='Not Pending'")['Profile']

In [ ]:
# Создание и запись словаря
def picks_stat(tmp):
  lst=[x.split('  ') for x in tmp['Bets']]
  lst=list(map(lambda x: x+[''] if len(x)==1  and ('-' in x[0] or '+' in x[0]) else x, lst))
  lst=list(map(lambda x: ['']+x if len(x)==1  and '.' in x[0] else x, lst))
  lst_teams=[x[0].split(' ')[0] for x in lst]
  lst_teams=list(filter(lambda x: x!='', lst_teams))
  lst_total=[x[1].split(' ')[0] for x in lst]
  lst_total=list(filter(lambda x: x!='', lst_total))
  t1=list(set(lst_teams))[0]
  t2=list(set(lst_teams))[1]

  print(f'{tmp.iloc[0,0]}. Всего: {len(tmp)}')
  print(f"Over: {lst_total.count('Over')}/{len(lst_total)}")
  print(f"Under: {lst_total.count('Under')}/{len(lst_total)}")

  print(f"{t1}: {lst_teams.count(t1)}/{len(lst_teams)}")
  print(f"{t2}: {lst_teams.count(t2)}/{len(lst_teams)}")
  print()
  tmp_dict={'Total': len(tmp),
            'Over': round(lst_total.count('Over')/len(lst_total),2),
            'Under': round(lst_total.count('Under')/len(lst_total),2),
            t1: round(lst_teams.count(t1)/len(lst_teams),2),
            t2: round(lst_teams.count(t2)/len(lst_teams),2)
  }
  return tmp_dict

picks=[]
today_picks=pd.DataFrame()
for pend in pending:
  tab = pd.read_html(www2+pend)
  today_picks = today_picks.append(tab[1])
today_picks=today_picks.drop([1,2,4,5], axis=1)
today_picks.columns=['Teams','Bets']
today_picks.head()
teams=list(set(today_picks['Teams']))
for t in teams:
  d=picks_stat(today_picks[today_picks.Teams==t])
  tt=t.replace('  ', ' at ')
  picks.append({t: d})

date=dt.now()
date=dt.strftime(date, '%y-%m-%d')
with open(path+f'KoC {date}.json', 'w') as f:
  json.dump(picks, f)

San Francisco  Philadelphia. Всего: 32
Over: 7/15
Under: 8/15
PHI: 28/31
SF: 3/31

St. Louis  Washington. Всего: 33
Over: 6/16
Under: 10/16
WAS: 24/31
STL: 7/31

LA Dodgers  Seattle. Всего: 31
Over: 5/15
Under: 10/15
SEA: 25/30
LAD: 5/30

Tampa Bay  Kansas City. Всего: 24
Over: 2/8
Under: 6/8
KC: 10/24
TB: 14/24

Texas  LA Angels. Всего: 21
Over: 3/9
Under: 6/9
LAA: 1/20
TEX: 19/20

Milwaukee  San Diego. Всего: 31
Over: 3/14
Under: 11/14
MIL: 22/30
SD: 8/30

Chi. White Sox  Boston. Всего: 29
Over: 3/14
Under: 11/14
CHW: 4/28
BOS: 24/28



In [ ]:
picks

[{'San Francisco at Philadelphia': {'Over': 0.47,
   'PHI': 0.9,
   'SF': 0.1,
   'Total': 32,
   'Under': 0.53}},
 {'St. Louis at Washington': {'Over': 0.38,
   'STL': 0.23,
   'Total': 33,
   'Under': 0.62,
   'WAS': 0.77}},
 {'LA Dodgers at Seattle': {'LAD': 0.17,
   'Over': 0.33,
   'SEA': 0.83,
   'Total': 31,
   'Under': 0.67}},
 {'Tampa Bay at Kansas City': {'KC': 0.42,
   'Over': 0.25,
   'TB': 0.58,
   'Total': 24,
   'Under': 0.75}},
 {'Texas at LA Angels': {'LAA': 0.05,
   'Over': 0.33,
   'TEX': 0.95,
   'Total': 21,
   'Under': 0.67}},
 {'Milwaukee at San Diego': {'MIL': 0.73,
   'Over': 0.21,
   'SD': 0.27,
   'Total': 31,
   'Under': 0.79}},
 {'Chi. White Sox at Boston': {'BOS': 0.86,
   'CHW': 0.14,
   'Over': 0.21,
   'Total': 29,
   'Under': 0.79}}]

# **АНАЛИЗ**

In [3]:
files=os.listdir(path)
files

['KoC 21-04-18.json', 'KoC 21-04-19.json']

In [4]:
with open(path+files[-1]) as f:
  tab=json.load(f)
tab

[{'San Francisco at Philadelphia': {'Over': 0.47,
   'PHI': 0.9,
   'SF': 0.1,
   'Total': 32,
   'Under': 0.53}},
 {'St. Louis at Washington': {'Over': 0.38,
   'STL': 0.23,
   'Total': 33,
   'Under': 0.62,
   'WAS': 0.77}},
 {'LA Dodgers at Seattle': {'LAD': 0.17,
   'Over': 0.33,
   'SEA': 0.83,
   'Total': 31,
   'Under': 0.67}},
 {'Tampa Bay at Kansas City': {'KC': 0.42,
   'Over': 0.25,
   'TB': 0.58,
   'Total': 24,
   'Under': 0.75}},
 {'Texas at LA Angels': {'LAA': 0.05,
   'Over': 0.33,
   'TEX': 0.95,
   'Total': 21,
   'Under': 0.67}},
 {'Milwaukee at San Diego': {'MIL': 0.73,
   'Over': 0.21,
   'SD': 0.27,
   'Total': 31,
   'Under': 0.79}},
 {'Chi. White Sox at Boston': {'BOS': 0.86,
   'CHW': 0.14,
   'Over': 0.21,
   'Total': 29,
   'Under': 0.79}}]

In [11]:
#Словарь {Матч: Тотал}
games_date='20'+files[-1][4:-5]
r=requests.get(f'https://www.covers.com/sports/mlb/matchups?selectedDate={games_date}', headers=headers)
s=BeautifulSoup(r.text, 'html.parser')
teams=s.find_all(class_='cmg_matchup_header_team_names')
noplay=s.find_all(class_='cmg_game_time')
teams=[x.get_text().replace('\r\n','').strip() for x in teams]
bs=pd.read_html(f'https://www.covers.com/sports/mlb/matchups?selectedDate={games_date}')
if len(noplay)>0:
  teams=teams[:-len(noplay)]
results={}
for c in range(len(teams)):
  if bs[c].loc[1,'R'] > bs[c].loc[0,'R']:
    fav_dog='-'
    if str(bs[c].loc[1,'ML'])[0]!='-':
      fav_dog='+'
    results[teams[c]]=[bs[c].loc[1,'Unnamed: 0'],  fav_dog, bs[c].loc[1,'R'] + bs[c].loc[0,'R'], bs[c].loc[1,'O/U']]
  else:
    fav_dog='-'
    if str(bs[c].loc[0,'ML'])[0]!='-':
      fav_dog='+'
    results[teams[c]]=[bs[c].loc[0,'Unnamed: 0'], fav_dog, bs[c].loc[1,'R'] + bs[c].loc[0,'R'], bs[c].loc[1,'O/U']]
display(results)
print()

# Подводим результаты picks
total_wins=0
winner_wins=0
picks_result={}
for d in tab:
  key=list(d.keys())[0]
  winner=results[key][0]
  if results[key][2] > results[key][3]:
    picks_result[key]={'winner': [d[key][winner], results[key][1]], 'Over': d[key]['Over']}
    if d[key]['Over'] > 0.5:
      total_wins += 1
    if d[key][winner] > 0.5:
      winner_wins += 1
  else:
    picks_result[key]={'winner': [d[key][winner], results[key][1]], 'Under': d[key]['Under']}
    if d[key]['Under'] > 0.5:
      total_wins += 1
    if d[key][winner] > 0.5:
      winner_wins += 1
print(f'ML: верных - {winner_wins} из {len(picks_result)}')
print(f'OU: верных - {total_wins} из {len(picks_result)}')
display(picks_result)

{'Chi. White Sox at Boston': ['BOS', '+', 15, 7.5],
 'LA Dodgers at Seattle': ['SEA', '+', 7, 8.5],
 'Milwaukee at San Diego': ['MIL', '+', 4, 6.5],
 'San Francisco at Philadelphia': ['SF', '-', 2, 8.5],
 'St. Louis at Washington': ['STL', '-', 17, 8.0],
 'Tampa Bay at Kansas City': ['TB', '-', 5, 7.5],
 'Texas at LA Angels': ['TEX', '+', 10, 8.5]}


ML: верных - 5 из 7
OU: верных - 4 из 7


{'Chi. White Sox at Boston': {'Over': 0.21, 'winner': [0.86, '+']},
 'LA Dodgers at Seattle': {'Under': 0.67, 'winner': [0.83, '+']},
 'Milwaukee at San Diego': {'Under': 0.79, 'winner': [0.73, '+']},
 'San Francisco at Philadelphia': {'Under': 0.53, 'winner': [0.1, '-']},
 'St. Louis at Washington': {'Over': 0.38, 'winner': [0.23, '-']},
 'Tampa Bay at Kansas City': {'Under': 0.75, 'winner': [0.58, '-']},
 'Texas at LA Angels': {'Over': 0.33, 'winner': [0.95, '+']}}

In [ ]:
# Подводим результаты picks
total_wins=0
winner_wins=0
picks_result={}
for d in tab:
  key=list(d.keys())[0]
  winner=results[key][0]
  if results[key][2] > results[key][3]:
    picks_result[key]={'winner': d[key][winner], 'Over': d[key]['Over']}
    if d[key]['Over'] > 0.5:
      total_wins += 1
    if d[key][winner] > 0.5:
      winner_wins += 1
  else:
    picks_result[key]={'winner': d[key][winner], 'Under': d[key]['Under']}
    if d[key]['Under'] > 0.5:
      total_wins += 1
    if d[key][winner] > 0.5:
      winner_wins += 1
display(total_wins, winner_wins, picks_result)

10

8

{'Arizona at Washington': {'Under': 0.56, 'winner': 0.71},
 'Atlanta at Chi. Cubs': {'Over': 0.2, 'winner': 0.8},
 'Baltimore at Texas': {'Under': 0.67, 'winner': 0.67},
 'Chi. White Sox at Boston': {'Under': 0.54, 'winner': 0.47},
 'Cleveland at Cincinnati': {'Over': 0.36, 'winner': 0.37},
 'Detroit at Oakland': {'Under': 0.77, 'winner': 0.33},
 'Houston at Seattle': {'Under': 0.69, 'winner': 0.65},
 'LA Dodgers at San Diego': {'Under': 0.62, 'winner': 0.75},
 'NY Mets at Colorado': {'Under': 0.27, 'winner': 0.19},
 'Pittsburgh at Milwaukee': {'Over': 0.6, 'winner': 0.61},
 'San Francisco at Miami': {'Under': 0.64, 'winner': 0.7},
 'St. Louis at Philadelphia': {'Under': 0.64, 'winner': 0.25},
 'Tampa Bay at NY Yankees': {'Under': 0.54, 'winner': 0.67},
 'Toronto at Kansas City': {'Under': 0.5, 'winner': 0.41}}

In [ ]:
d[key]['PHI']

0.25

In [ ]:
tmp=today_picks[today_picks.Teams=='Baltimore  Texas']
lst=[x.split('  ') for x in tmp['Bets']]
lst=list(map(lambda x: x+[''] if len(x)==1  and ('-' in x[0] or '+' in x[0]) else x, lst))
lst=list(map(lambda x: ['']+x if len(x)==1  and '.' in x[0] else x, lst))
lst_teams=[x[0].split(' ')[0] for x in lst]
lst_teams=list(filter(lambda x: x!='', lst_teams))
lst_total=[x[1].split(' ')[0] for x in lst]
lst_total=list(filter(lambda x: x!='', lst_total))
t1=list(set(lst_teams))[0]
t2=list(set(lst_teams))[1]

print(f'{tmp.iloc[0,0]}. Всего: {len(tmp)}')
print(f"Over: {lst_total.count('Over')}/{len(lst_total)}")
print(f"Under: {lst_total.count('Under')}/{len(lst_total)}")

print(f"{t1}: {lst_teams.count(t1)}/{len(lst_teams)}")
print(f"{t2}: {lst_teams.count(t2)}/{len(lst_teams)}")